In [1]:
import pandas as pd
import numpy as np
import mysql.connector
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
db_connection = mysql.connector.connect(
    host='ec2-44-223-143-62.compute-1.amazonaws.com',
    user='nats',
    password='nats',
    database='bookmate'
)

In [4]:
users_df = pd.read_sql('SELECT * FROM users', con=db_connection)
library_df = pd.read_sql('SELECT * FROM library', con=db_connection)
books = pd.read_sql('SELECT * FROM top_books', con=db_connection)

C:\Users\hiaru\AppData\Local\Temp\ipykernel_34540\3219696273.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  users_df = pd.read_sql('SELECT * FROM users', con=db_connection)
C:\Users\hiaru\AppData\Local\Temp\ipykernel_34540\3219696273.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  library_df = pd.read_sql('SELECT * FROM library', con=db_connection)
C:\Users\hiaru\AppData\Local\Temp\ipykernel_34540\3219696273.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  books = pd.read_sql('SELECT * FROM top_books', con=db_co

In [5]:
db_connection.close()

In [6]:
users_df

,id,LibID,BookmateID,email,first_name,last_name,nickname,phone_num,instagram,picture_url,instagram_public,phone_public,email_public,opted_in,profile_done,created_at


In [7]:
users_df = users_df[users_df['opted_in'] != 0]

In [7]:
users_dfc

,id,LibID,BookmateID,email,first_name,last_name,nickname,phone_num,instagram,picture_url,instagram_public,phone_public,email_public,opted_in,profile_done,created_at


In [8]:
library_df

,LibID,id,Fave_Books,Wish_List,answers
0,LIB-20240706102626-a4f737,108917069924969544799,None,None,None
1,LIB-20240706103025-cdac69,104476313118646525922,None,None,None


In [8]:
books

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,num_ratings,avg_rating
0,901,000649840X,9.0,Angelas Ashes,Frank Mccourt,0,Harpercollins Uk,http://images.amazon.com/images/P/000649840X.0...,http://images.amazon.com/images/P/000649840X.0...,http://images.amazon.com/images/P/000649840X.0...,None,None
1,11676,0006547834,5.0,Miss Smillas Feeling for Snow,Peter Hoeg,0,Flamingo,http://images.amazon.com/images/P/0006547834.0...,http://images.amazon.com/images/P/0006547834.0...,http://images.amazon.com/images/P/0006547834.0...,None,None
2,13664,000654861X,10.0,Life and Death in Shanghai,Nien Cheng,1995,HarperCollins Publishers,http://images.amazon.com/images/P/000654861X.0...,http://images.amazon.com/images/P/000654861X.0...,http://images.amazon.com/images/P/000654861X.0...,None,None
3,14387,0006550576,0.0,Red Leaves,Paullina Simons,0,Flamingo,http://images.amazon.com/images/P/0006550576.0...,http://images.amazon.com/images/P/0006550576.0...,http://images.amazon.com/images/P/0006550576.0...,None,None
4,11676,0006550789,7.0,253,Geoff Ryman,0,Flamingo,http://images.amazon.com/images/P/0006550789.0...,http://images.amazon.com/images/P/0006550789.0...,http://images.amazon.com/images/P/0006550789.0...,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...
7507,11676,8401328810,6.0,Tan Veloz Como El Deseo,Laura Esquivel,2001,"Plaza &amp; Janes Editores, S.A.",http://images.amazon.com/images/P/8401328810.0...,http://images.amazon.com/images/P/8401328810.0...,http://images.amazon.com/images/P/8401328810.0...,None,None
7508,3675,8433925180,0.0,El Antropologo Inocente,Nigel Barley,1998,Anagrama,http://images.amazon.com/images/P/8433925180.0...,http://images.amazon.com/images/P/8433925180.0...,http://images.amazon.com/images/P/8433925180.0...,None,None
7509,21484,8478884459,0.0,Harry Potter y la piedra filosofal,J. K. Rowling,1999,Lectorum Publications,http://images.amazon.com/images/P/8478884459.0...,http://images.amazon.com/images/P/8478884459.0...,http://images.amazon.com/images/P/8478884459.0...,None,None
7510,276925,950491036X,10.0,La Sombra del Viento,Carlos Ruiz Zafon,2003,Planeta,http://images.amazon.com/images/P/950491036X.0...,http://images.amazon.com/images/P/950491036X.0...,http://images.amazon.com/images/P/950491036X.0...,None,None


In [102]:
pt = pd.read_pickle('pivot_table.pkl')
similarity_scores = np.load('similarity_scores.npy')

In [103]:
def calculate_library_similarity(lib1, lib2):
    lib1_books = lib1.split(',')
    lib2_books = lib2.split(',')
    
    print("Lib1")
    print(lib1_books)
    
    print("Lib2")
    print(lib2_books)
    
    sim_scores = []
    for book1 in lib1_books:
        index1 = np.where(books['ISBN'] == book1)[0]
        if len(index1) == 0:
            print(f"Book {book1} not found in books DataFrame.")
            continue
        index1 = index1[0]
        
        for book2 in lib2_books:
            index2 = np.where(books['ISBN'] == book2)[0]
            if len(index2) == 0:
                print(f"Book {book2} not found in books DataFrame.")
                continue
            index2 = index2[0]
            
            print(index1)
            print(index2)
            print(similarity_scores[index1][index2])
            sim_scores.append(similarity_scores[index1][index2])
    
    if sim_scores:
        return np.mean(sim_scores)
    else:
        return 0


In [89]:
def calculate_answer_similarity(ans1, ans2):
    if len(ans1) != len(ans2):
        raise ValueError("Answer strings must be of the same length.")
    
    matches = sum(1 for a, b in zip(ans1, ans2) if a == b)
    return matches / len(ans1)

In [90]:
def pair_users(users_df, library_df):
    user_similarities = {}
    
    for i, user1 in users_df.iterrows():
        lib1 = library_df[library_df['id'] == user1['id']]['Fave_Books'].values[0]
        ans1 = library_df[library_df['id'] == user1['id']]['answers'].values[0]
        
        for j, user2 in users_df.iterrows():
            if i >= j:
                continue
            
            lib2 = library_df[library_df['id'] == user2['id']]['Fave_Books'].values[0]
            ans2 = library_df[library_df['id'] == user2['id']]['answers'].values[0]
            
            lib_similarity = calculate_library_similarity(lib1, lib2)
            ans_similarity = calculate_answer_similarity(ans1, ans2)
            
            overall_similarity = (lib_similarity + ans_similarity) / 2
            user_similarities[(user1['id'], user2['id'])] = overall_similarity
    
    sorted_pairs = sorted(user_similarities.items(), key=lambda x: x[1], reverse=True)
    
    paired_users = []
    used_users = set()
    
    for (user1, user2), similarity in sorted_pairs:
        if user1 not in used_users and user2 not in used_users:
            paired_users.append((user1, user2, similarity))
            used_users.add(user1)
            used_users.add(user2)
    
    return paired_users

In [116]:
paired_users = pair_users(users_df, library_df)

Lib1
['0451176464', '0451132378', '0451137965', '0451125452']
Lib2
['0345391810', '0451161351', '0345385764']
1138
6
0.0
1138
2893
0.0
1138
26
0.0
534
6
0.0
534
2893
0.0
534
26
0.0
173
6
0.0
173
2893
0.0
173
26
0.0
2123
6
0.0
2123
2893
0.0
2123
26
0.0


ValueError: Answer strings must be of the same length.

In [105]:
for user1, user2, similarity in paired_users:
    print(f"Paired User {user1} with User {user2} (Similarity: {similarity:.2f})")

Paired User 104237751557836536880 with User 114586263119569323859 (Similarity: 0.46)
Paired User 104476313118646525922 with User 108917069924969544799 (Similarity: 0.19)
